# Merge sensor data with MET lables

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np


os.chdir("/home/cernerrood246/University/DataMining")
pd.set_option("display.precision", 2)

In [ ]:
participants_list = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17']

## Load Datasets

### Load MET

In [ ]:
MET_all_list = [pd.read_pickle(f"preprocessed_dataset/{participant}/MET.pkl") for participant in participants_list]
MET_all_list[1]


### Load Demographics


In [ ]:
demographics_df = pd.read_csv("original_dataset/Demographics.csv")
#dummy encoding Gender
demographics_df = pd.get_dummies(demographics_df, columns = ["Gender"], dtype=int)
demographics_df = demographics_df.drop(columns=["Participant", "Comments"])
demographics_df.to_pickle("preprocessed_dataset/demographics.pkl")

### Study Information

In [43]:
study_information = pd.read_csv("original_dataset/Study_Information.csv")

#convert Start_Sit	Start_Stand	Start_Cycle1	Start_Cycle2	Start_Run1	Start_Run2 to datetime
study_information["Start_Sit"] = pd.to_datetime(study_information["Start_Sit"])
study_information["Start_Stand"] = pd.to_datetime(study_information["Start_Stand"])
study_information["Start_Cycle1"] = pd.to_datetime(study_information["Start_Cycle1"])
study_information["Start_Cycle2"] = pd.to_datetime(study_information["Start_Cycle2"])
study_information["Start_Run1"] = pd.to_datetime(study_information["Start_Run1"])
study_information["Start_Run2"] = pd.to_datetime(study_information["Start_Run2"])

def assign_activity(row):
    """returns activity, speed and met for row

    :param row: _description_
    :return: _description_
    """    
    if row.name < row["Start_Stand"]:
        return {"activity_suggested" : 1, "speed": 0, "met_suggested" : 1}
    if row["Start_Stand"] <= row.name < row["Start_Cycle1"]:
        return {"activity_suggested" : 1, "speed": 0, "met_suggested" : 1.2}
    if row["Start_Cycle1"] <= row.name < row["Start_Cycle2"]:
        return {"activity_suggested" : 2, "speed": row["Cycle_Speed1"], "met_suggested" : row["MET_Cycle1"]}
    if row["Start_Cycle2"] <= row.name < row["Start_Run1"]:
        return {"activity_suggested" : 2, "speed": row["Cycle_Speed2"], "met_suggested" : row["MET_Cycle2"]}
    if row["Start_Run1"] <= row.name < row["Start_Run2"]:
        return {"activity_suggested" : 5, "speed": row["Run_Speed1"], "met_suggested" : row["MET_Run1"]}
    if row["Start_Run2"] <= row.name:
        return {"activity_suggested" : 5, "speed": row["Run_Speed2"], "met_suggested" : row["MET_Run2"]}
    return {"activity_suggested" : None, "speed": None, "met_suggested" : None}



study_information

,Participant,Start_Sit,Start_Stand,Start_Cycle1,Start_Cycle2,Start_Run1,Start_Run2,Cycle_Speed1,Cycle_Speed2,Run_Speed1,Run_Speed2,Comments,MET_Sit,MET_Stand,MET_Cycle1,MET_Cycle2,MET_Run1,MET_Run2
0,P01,2021-12-03 16:58:50,2021-12-03 17:03:00,2021-12-03 17:08:00,2021-12-03 17:13:00,2021-12-03 17:18:00,2021-12-03 17:23:00,14,20,4.0,6.0,,1,1.2,10,16,4.5,10.0
1,P02,2021-11-25 17:17:00,2021-11-25 17:23:00,2021-11-25 17:36:10,2021-11-25 17:41:50,2021-11-25 17:46:50,2021-11-25 17:51:00,14,19,5.0,8.5,Muse headband data lost due to connectivity is...,1,1.2,10,12,8.0,14.0
2,P03,2021-11-26 16:20:20,2021-11-26 16:26:20,2021-11-26 16:53:06,2021-11-26 16:56:18,2021-11-26 17:00:13,2021-11-26 17:04:05,18,22,6.0,7.0,Second part V02 data got losT,1,1.2,12,16,10.0,11.5
3,P04,2021-11-26 18:15:48,2021-11-26 18:21:56,2021-11-26 18:27:00,2021-11-26 18:32:20,2021-11-26 18:37:10,2021-11-26 18:45:00,15,22,6.0,9.0,Interruption at 18:41,1,1.2,10,16,10.0,15.0
4,P05,2021-11-29 09:40:47,2021-11-29 09:45:40,2021-11-29 09:51:41,2021-11-29 09:56:41,2021-11-29 10:02:10,2021-11-29 10:06:40,15,22,5.0,7.0,,1,1.2,10,16,8.0,11.5
5,P06,2021-11-29 10:37:54,2021-11-29 10:43:00,2021-11-29 10:49:20,2021-11-29 10:54:35,2021-11-29 11:00:00,2021-11-29 11:06:00,14,20,5.0,7.0,VO2 data was lost (?),1,1.2,10,16,8.0,11.5
6,P07,2021-11-29 15:33:02,2021-11-29 15:38:15,2021-11-29 15:45:25,2021-11-29 15:50:00,2021-11-29 15:57:00,2021-11-29 16:02:00,16,23,5.5,8.0,Low quality HR during running,1,1.2,12,16,9.0,13.5
7,P08,2021-11-29 18:00:31,2021-11-29 18:05:30,2021-11-29 18:11:00,2021-11-29 18:16:00,2021-11-29 18:21:00,2021-11-29 18:26:00,9,18,5.0,9.0,,1,1.2,4,12,8.0,15.0
8,P09,2021-11-30 09:19:12,2021-11-30 09:24:20,2021-11-30 09:30:30,2021-11-30 09:36:00,2021-11-30 09:41:20,2021-11-30 09:46:20,14,30,6.4,9.0,,1,1.2,10,16,11.0,15.0
9,P10,2021-11-30 10:32:45,2021-11-30 10:37:40,2021-11-30 11:45:00,2021-11-30 11:50:20,2021-11-30 11:55:30,2021-11-30 12:01:00,20,24,5.0,6.5,Sleepiness level 2 if 4 is very sleepy,1,1.2,16,16,8.0,11.0


### Load Fitbit

In [ ]:
fitbit_path = "Fitbit_data/average_data_5S_new/average_data_5S_new"
fitbit_df_list = []
for df in os.listdir(fitbit_path):
    temp_df = pd.read_csv(f"{fitbit_path}/{df}")
    #convert index to datetime
    temp_df = temp_df.rename(columns={"Unnamed: 0": "Time"})
    temp_df["Time"] = pd.to_datetime(temp_df["Time"])

    temp_df = temp_df.set_index("Time")
    #upsample to 1 second without filling
    temp_df = temp_df.resample("1S").asfreq().copy()

    #fill next 4 values with the previous value
    temp_df = temp_df.ffill(limit=4)
    temp_df = temp_df[temp_df['value__bpm'].notna()]

    
    fitbit_df_list.append(temp_df)
    
fitbit_df_all = pd.concat(fitbit_df_list, axis=0)
fitbit_df_all 


### Load Apple Watch

In [ ]:
apple_path = "Apple_watch/Active_energy_apple_watch_processed.csv"

apple_df_all = pd.read_csv(apple_path)
#convert index to datetime
apple_df_all = apple_df_all.rename(columns={"date": "Time"})
apple_df_all["Time"] = pd.to_datetime(apple_df_all["Time"])
#remove timezone 
apple_df_all["Time"] = apple_df_all["Time"].dt.tz_localize(None)

apple_df_all = apple_df_all.set_index("Time")

type(apple_df_all.index[0])

### Load E4

In [ ]:
E4_all_list = [pd.read_pickle(f"merged_dataset/{participant}/E4.pkl") for participant in participants_list]


In [ ]:
E4_all_list[1]["Delta_Time"]

## Merge Datasets

In [45]:
merged_df_list = []
for participant_number, met_df in enumerate(MET_all_list):
    #left join on met_df
    temp_df = met_df.merge(fitbit_df_all, how="left", left_index=True, right_index=True)
    temp_df = temp_df.merge(E4_all_list[participant_number].drop(columns = {"Delta_Time", "MET"}), how="left", left_on = "Time[s]", right_on = "Time[s]")
    temp_df.index = met_df.index
    temp_df = temp_df.merge(apple_df_all, how="left", left_index=True, right_index=True)

    #add demographics cross join
    temp_df = temp_df.merge(demographics_df.iloc[participant_number].to_frame().T, how="cross")

    #add study information
    temp_df = temp_df.merge(study_information.iloc[participant_number].to_frame().T, how="cross")


    temp_df.index = met_df.index

    temp_df[["activity_suggested", "speed", "met_suggested"]] = temp_df.apply(lambda row: assign_activity(row), axis=1, result_type='expand')
    merged_df_list.append(temp_df)

merged_df_list[0]

,Time[s],Delta_Time,MET,Activity,value__bpm,value__confidence,X,Y,Z,Magnitude,...,Comments,MET_Sit,MET_Stand,MET_Cycle1,MET_Cycle2,MET_Run1,MET_Run2,activity_suggested,speed,met_suggested
Time,,,,,,,,,,,,,,,,,,,,,
2021-12-03 16:58:51,1.0,0 days 00:00:00,1.31,1.0,91.0,1.0,30.0,0.0,55.67,63.24,...,,1,1.2,10,16,4.5,10.0,1.0,0.0,1.0
2021-12-03 16:58:52,2.0,0 days 00:00:01,1.31,1.0,91.0,1.0,30.0,0.0,55.75,63.31,...,,1,1.2,10,16,4.5,10.0,1.0,0.0,1.0
2021-12-03 16:58:53,3.0,0 days 00:00:02,1.52,1.0,91.0,1.0,30.0,0.0,55.80,63.35,...,,1,1.2,10,16,4.5,10.0,1.0,0.0,1.0
2021-12-03 16:58:54,4.0,0 days 00:00:03,1.63,1.0,91.0,1.0,30.0,0.0,55.60,63.18,...,,1,1.2,10,16,4.5,10.0,1.0,0.0,1.0
2021-12-03 16:58:55,5.0,0 days 00:00:04,1.56,1.0,88.0,1.0,30.0,0.0,55.80,63.35,...,,1,1.2,10,16,4.5,10.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-03 17:28:48,1798.0,0 days 00:29:57,5.32,5.0,139.0,1.0,20.2,-49.4,36.00,64.38,...,,1,1.2,10,16,4.5,10.0,5.0,6.0,10.0
2021-12-03 17:28:49,1799.0,0 days 00:29:58,5.11,5.0,139.0,1.0,18.0,-50.2,33.80,63.14,...,,1,1.2,10,16,4.5,10.0,5.0,6.0,10.0
2021-12-03 17:28:50,1800.0,0 days 00:29:59,4.46,5.0,142.0,2.0,17.8,-50.2,34.20,63.30,...,,1,1.2,10,16,4.5,10.0,5.0,6.0,10.0


In [46]:
for i, participant_df in enumerate(merged_df_list):
    # os.makedirs(f"merged_dataset/P{i+1}", exist_ok=True)
    participant_df.to_pickle(f"merged_dataset/P{i+1}/ALL.pkl")
    #os.remove(f"preprocessed_dataset/P{i+1}/MET.csv")

In [ ]:
merged_df_list[1].hist(column = "Time[s]", bins = 100)
merged_df_list[1]

In [ ]:
for df in merged_df_list:
    fig, ax1 = plt.subplots(figsize=(20, 10))
    ax2 = ax1.twinx()
    ax3 = ax1.twinx()
    ax4 = ax1.twinx()

    # plot MET on the first y-axis
    ax1.plot(df.index, df["MET"], 'g-')
    ax1.set_ylabel('MET', color='g')

    # plot value_bpm on the second y-axis
    ax2.plot(df.index, df["value__bpm"], 'b-')
    ax2.set_ylabel('BPM', color='b')

    # plot Magnitude on the third y-axis
    ax3.plot(df.index, df["Magnitude"], 'r-')
    ax3.set_ylabel('Magnitude', color='r')

    #plot active energy (kJ) on the fourth y-axis
    ax4.plot(df.index, df["active energy (kJ)"], 'y-')
    ax4.set_ylabel('Active Energy', color='y')

    # adjust the position of the third y-axis
    ax3.spines['right'].set_position(('outward', 60))
    ax3.set_ylim(0, df["Magnitude"].max() * 1.1)
    ax3.set_ylabel('Magnitude', color='r')

    ax1.set_xlabel('Time')

    plt.show()